In [2]:
import time
import csv
import json
import random
import time
import Levenshtein
import requests
import wikitextparser as wtp
from wiki_dump_reader import iterate
from tqdm import tqdm
from SPARQLWrapper import SPARQLWrapper, JSON
from statistics import mean

from extraction_croissement import *

In [4]:
# croisement avec DBpedia
with open('authors_ls.json', 'r') as input_file:
    author_ls = json.load(input_file)

with open('DBpedia/ecrivains_dbpedia_fr.txt', 'r') as input_file:
    DBpedia_authors = []
    for line in input_file:
        DBpedia_authors.append(line.split(';')[-1])

found_once = 0
count = 0
count_freq = {}
count_info = 0

couple_writers = []
data_base_stats = {}

for author_DB in tqdm(author_ls, total=len(author_ls)):
    found = False
    count_freq_i = 0
    for author_DBpedia in DBpedia_authors:
        normalised_DBpedia_title = normalize(author_DBpedia)
        dist_auteur = Levenshtein.distance(author_DB, normalised_DBpedia_title)
        if max(1, min(len(author_DB), len(normalised_DBpedia_title)) / 6) > dist_auteur > max(1, min(len(author_DB), len(normalised_DBpedia_title)) / 8):
            print("Autheur DB: ", author_DB + " " * (40 - len(author_DB)), "Autheur DBpedia: ", normalised_DBpedia_title)
        if dist_auteur < max(1, min(len(author_DB), len(normalised_DBpedia_title)) / 8):
            new_couple = {
                        'author DB': author_DB,
                        'author wikidata': author_DBpedia,
                        'book DB': author_ls[author_DB]
                    }
            count += 1
            count_freq_i += 1
            found = True

            couple_writers.append(new_couple)
            for book in author_ls[author_DB]:
                try:
                    data_base_stats[book['data_base']] += 1
                except:
                    data_base_stats[book['data_base']] = 1

print("nombre d'autheurs en commun: ", found_once, " soit ", found_once * 100 / len(list(author_ls)), "%")
print("nombre d'autheurs reconnus: ", count, " soit ", count * 100 / len(list(author_ls)), "%")
print("Nombre d'information catégorisée: ", count_info, " soit ", count_info * 100 / len(list(author_ls)), "%")
print("Frequence de reconnaissance: ", count_freq)
print("repartition dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats))

with open('DBpedia/couple_authors_DBpedia.json', 'w') as outfile:
    json.dump(couple_writers, outfile)

100%|██████████| 20059/20059 [04:07<00:00, 81.11it/s]


Autheur DB:  anita mestokosho                         Autheur DBpedia:  rita mestokosho
Autheur DB:  michel levesque                          Autheur DBpedia:  michel j levesque
Autheur DB:  fernand ouellette                        Autheur DBpedia:  fernand ouellet
Autheur DB:  gilles henault                           Autheur DBpedia:  gilles devault
Autheur DB:  sonia laflamme                           Autheur DBpedia:  sonia k laflamme
Autheur DB:  dany tremblay                            Autheur DBpedia:  tony tremblay
Autheur DB:  michel poulin                            Autheur DBpedia:  michel gaulin
Autheur DB:  sylvain houde                            Autheur DBpedia:  sylvain hotte
Autheur DB:  francine amour                           Autheur DBpedia:  francine d amour
Autheur DB:  sylvie tremblay                          Autheur DBpedia:  sylvaine tremblay
Autheur DB:  serge bouchard                           Autheur DBpedia:  serge boucher
Autheur DB:  serge bouchard        

In [10]:
sparql = SPARQLWrapper("http://fr.dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT DISTINCT ?s
    WHERE {
        {{
            ?s dcterms:subject ?cat .
            ?cat skos:broader* <http://fr.dbpedia.org/resource/Catégorie:Écrivain_québécois>
        }}
        UNION {{
            ?s
        }}.
    }
""")

sparql.setReturnFormat(JSON)
results_dbpedia_fr = sparql.query().convert()

In [7]:
sparql = SPARQLWrapper("http://fr.dbpedia.org/sparql")
sparql.setQuery("""
    SELECT DISTINCT ?hasprop ?v
    where { <http://fr.dbpedia.org/resource/Richard_Garneau> ?hasprop ?v}
""")

sparql.setReturnFormat(JSON)
results_dbpedia_fr = sparql.query().convert()

In [5]:
with open('DBpedia/ecrivains_dbpedia_fr.txt', 'r') as input_file:
    DBpedia_authors_uri = []
    for line in input_file:
        DBpedia_authors_uri.append(line.split(';')[0])

sparql = SPARQLWrapper("http://fr.dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

result = {}

for author_uri in DBpedia_authors_uri:
    time.sleep(1)
    sparql.setQuery("""
    SELECT DISTINCT ?hasprop ?v
    where {{ {0} ?hasprop ?v}}
""".format("<" + author_uri.replace(" ", "") + ">"))
    results_dbpedia_fr = sparql.query().convert()
    result[author_uri.replace(" ", "")] = [(duo['hasprop']['value'], duo['v']['value']) for duo in results_dbpedia_fr['results']['bindings']]

In [13]:
prop_dict = {}
prop_counter = {}
for author_uri, properties in result.items():
    for property in properties:
        try:
            prop_counter[property[0]] += 1
        except:
            prop_counter[property[0]] = 1

for author_uri, properties in result.items():
    for rep_prop in prop_counter:
        counter = 0
        for property in properties:
            if property[0] == rep_prop:
                counter += 1
        try:
            prop_dict[rep_prop].append(counter)
        except:
            prop_dict[rep_prop] = [counter]

prop_resume = {}
for rep_prop in prop_dict:
    prop_resume[rep_prop] = {
        'non_null_mean': mean([e for e in prop_dict[rep_prop] if e]),
        'nb_non_null': sum([1 for e in prop_dict[rep_prop] if e]),
        'sum': sum(prop_dict[rep_prop]),
    }

# print(json.dumps(sorted(prop_resume.items(), key=lambda item: item[1]['nb_non_null'], reverse=True), indent=1))

with open('DBpedia/list_properies_by_author.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['name', 'non_null_mean', 'nb_non_null', 'sum' ])
        writer.writeheader()
        for rep_prop_name, rep_prop_dict in prop_resume.items():
            rep_prop_dict['name'] = rep_prop_name
            writer.writerow(rep_prop_dict)